In [1]:
import ffmpeg
import numpy as np
import cv2
import mediapipe as mp
import os
import sys

def extract_frames_without_hands_ffmpeg(input_video_path, output_folder):
    """
    Extracts and saves all frames from the input video that do not contain any detected hands.

    :param input_video_path: Path to the input video file.
    :param output_folder: Directory where extracted frames will be saved.
    """
    # Ensure FFmpeg is installed
    try:
        ffmpeg_version = ffmpeg.probe('')
    except ffmpeg.Error as e:
        print("FFmpeg is not installed or not found in PATH.")
        sys.exit(1)

    # Create the output directory if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Initialize MediaPipe Hands
    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands(
        static_image_mode=False,
        max_num_hands=2,
        min_detection_confidence=0.3
    )

    # Probe the video to get video stream information
    try:
        probe = ffmpeg.probe(input_video_path)
    except ffmpeg.Error as e:
        print(f"FFmpeg probe error: {e.stderr.decode()}")
        sys.exit(1)

    # Extract video stream information
    video_streams = [stream for stream in probe['streams'] if stream['codec_type'] == 'video']
    if not video_streams:
        print("No video streams found in the input file.")
        sys.exit(1)

    video_stream = video_streams[0]
    width = int(video_stream['width'])
    height = int(video_stream['height'])
    total_frames = int(video_stream.get('nb_frames', 0))

    print(f"Video Resolution: {width}x{height}")
    print(f"Total Frames: {total_frames if total_frames else 'Unknown'}")

    # Set up FFmpeg process to read raw video frames
    process = (
        ffmpeg
        .input(input_video_path)
        .output('pipe:', format='rawvideo', pix_fmt='rgb24')
        .run_async(pipe_stdout=True, pipe_stderr=True)
    )

    frame_size = width * height * 3  # 3 bytes per pixel for RGB
    frame_count = 0
    saved_count = 0

    try:
        while True:
            in_bytes = process.stdout.read(frame_size)
            if not in_bytes:
                break  # End of video

            frame = (
                np
                .frombuffer(in_bytes, np.uint8)
                .reshape([height, width, 3])
            )

            frame_count += 1

            # Convert frame to BGR for OpenCV compatibility
            frame_bgr = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

            # Process frame with MediaPipe to detect hands
            results = hands.process(cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB))

            # If no hands are detected, save the frame
            if not results.multi_hand_landmarks:
                output_path = os.path.join(output_folder, f"frame_{frame_count:06d}.png")
                cv2.imwrite(output_path, frame_bgr)
                saved_count += 1
                print(f"Saved frame {frame_count} to {output_path}")

            # Optional: Display progress every 100 frames
            if frame_count % 100 == 0:
                progress = (frame_count / total_frames * 100) if total_frames else 'Unknown'
                print(f"Processed {frame_count} frames. Progress: {progress}%")

    except KeyboardInterrupt:
        print("Process interrupted by user.")
    finally:
        process.stdout.close()
        process.wait()
        hands.close()

    print("\nExtraction Completed.")
    print(f"Total Frames Processed: {frame_count}")
    print(f"Total Frames Without Hands Saved: {saved_count}")
    print(f"Frames saved in directory: {output_folder}")

if __name__ == "__main__":
    # Example usage
    input_video_path = '/Users/emin/Desktop/ee475/piano-transcription/dataset/MIDItest/miditest_videos/5.mp4'  # Update this path as needed
    output_folder = 'dataset/MIDItest/frames_without_hands'

    # Verify input video exists
    if not os.path.isfile(input_video_path):
        print(f"Input video file does not exist: {input_video_path}")
        sys.exit(1)

    extract_frames_without_hands_ffmpeg(input_video_path, output_folder)


FFmpeg is not installed or not found in PATH.


SystemExit: 1

/Users/emin/Desktop/ee475/piano-transcription/.venv/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
